In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import pipeline, preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import model_selection, metrics

from df_transformers import SelectColumnsTransfomer

%matplotlib inline

In [2]:
train_data = pd.read_csv("data/train.csv", index_col="PassengerId")
test_data = pd.read_csv("data/test.csv", index_col="PassengerId")
print(f"Shape of train data: {train_data.shape}. Shape of test data: {test_data.shape}")

Shape of train data: (891, 11). Shape of test data: (418, 10)


In [3]:
columns_to_remove = ['Name', 'Ticket']
train_data = train_data.drop(columns_to_remove, axis=1)
test_data = test_data.drop(columns_to_remove, axis=1)

### Заполняем пропущенные значения - ДОРАБОТАТЬ, ПЕРЕНЕСТИ В ПАЙПЛАЙН

In [4]:
train_data[['Cabin', 'Embarked']] = train_data[['Cabin', 'Embarked']].fillna("NAN")
test_data[['Cabin', 'Embarked']] = test_data[['Cabin', 'Embarked']].fillna("NAN")

mean_age = train_data['Age'].mean()
median_age = train_data['Age'].median()
print(f'Среднее значение возраста: {mean_age}. Медиана: {median_age}')

train_data[['Age']] = train_data[['Age']].fillna(median_age)
test_data[['Age']] = test_data[['Age']].fillna(median_age)

mean_fare = test_data['Fare'].mean()
test_data[['Fare']] = test_data[['Fare']].fillna(mean_fare)

Среднее значение возраста: 29.69911764705882. Медиана: 28.0


### Разделяем выборку на трейн и тест

In [5]:
X = train_data.drop(['Survived'], axis=1)
y = train_data['Survived']
X.shape

(891, 8)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [7]:
numerical_features = ['Age', 'SibSp', 'Parch', 'Fare']
cat_features = X.columns.drop(numerical_features).tolist()

### Собираем пайплайн

In [8]:
# Для выбора столбцов можно воспользоваться ColumnTransformer

num_pipeline = make_pipeline(
    SelectColumnsTransfomer(numerical_features),
    preprocessing.StandardScaler(with_mean = 0)
)

cat_pipeline = make_pipeline(
    SelectColumnsTransfomer(cat_features),
    preprocessing.OneHotEncoder(handle_unknown = 'ignore')
)

preprocessing_features = pipeline.FeatureUnion(transformer_list = [        
            ('numeric_variables_processing', num_pipeline),
            ('categorical_variables_processing', cat_pipeline),
        ])

### Grid search

In [9]:
# parameteres = {'SVM__C':[0.001,0.1,10,100,10e5], 'SVM__gamma':[0.1,0.01]}
# grid = GridSearchCV(pipeline, param_grid=parameteres, cv=5)
# grid.fit(X_train, y_train)
# print "score = %3.2f" %(grid.score(X_test,y_test))
# print grid.best_params_

### Define a cross validation strategy

In [10]:
def accuracy_cv(model):
    accuracies = cross_val_score(model, X_train, y_train, scoring="accuracy", cv=5)
    return(accuracies)

## Models

### Логистическая регрессия

In [11]:
lr = LogisticRegression()

In [12]:
lr_pipeline = make_pipeline(preprocessing_features, lr)

In [13]:
print(f"The mean accuracy is {accuracy_cv(lr_pipeline).mean()}, std: {accuracy_cv(lr_pipeline).std()}")

/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


The mean accuracy is 0.7962394265232975, std: 0.016871010978027283


/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:451: DataConversionW

Last result: The accuracy is 0.7930265232974911, std: 0.0174907006254278

### Random Forest classifaer

In [14]:
rfc = RandomForestClassifier(n_estimators = 300)

In [15]:
rfc_pipeline = make_pipeline(preprocessing_features, rfc)

In [16]:
print(f"The mean accuracy is {accuracy_cv(rfc_pipeline).mean()}, std: {accuracy_cv(rfc_pipeline).std()}")

/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/base.py:467: DataCo

The mean accuracy is 0.794536610343062, std: 0.009219854914637839


/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


Last result: The mean accuracy is 0.7865105990783411, std: 0.019310961490566207

### SVC

In [17]:
svc = SVC()

In [18]:
svc_pipeline = make_pipeline(preprocessing_features, svc)

In [19]:
print(f"The mean accuracy is {accuracy_cv(svc_pipeline).mean()}, std: {accuracy_cv(svc_pipeline).std()}")

/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transfor

The mean accuracy is 0.7930644137224783, std: 0.025854206966768677


/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transfo

### KNeighborsClassifier

In [20]:
knn = KNeighborsClassifier(n_neighbors = 3)

In [21]:
knn_pipeline = make_pipeline(preprocessing_features, knn)

In [22]:
print(f"The mean accuracy is {accuracy_cv(knn_pipeline).mean()}, std: {accuracy_cv(knn_pipeline).std()}")

/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/base.py:467: DataCo

The mean accuracy is 0.7816462877624166, std: 0.01989377581457177


/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/tarasovnm/anaconda3/lib/python3.7/site-packages/sklearn/base.py:467: DataCo

## Test data submission

In [23]:
predicted = lr_pipeline.predict(test_data)
test_data["Survived"] = predicted
test_data["Survived"].to_csv('logistic_regression.csv', header=True)

NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [ ]:
test_data["Survived"].head()